# Setup

In [1]:
# Setup
import pandas as pd
import numpy as np
from pathlib import Path
import yfinance as yf
from datetime import timedelta
import time

output_dir = Path("../data/processed")
output_dir.mkdir(parents=True, exist_ok=True)


# IPO Data Collection: Curated Tech IPO List

Manual compilation from public sources (2018-2024)


In [2]:
# Curated list of major tech IPOs 2018-2024
tech_ipos_data = {
    'Company': [
        # 2024
        'Reddit', 'Astera Labs', 'Rubrik', 'Ibotta', 'Tempus AI',
        
        # 2023  
        'Arm Holdings', 'Instacart', 'Klaviyo', 'Birkenstock', 'Nextracker',
        
        # 2022
        'TPG', 'Credo Technology',
        
        # 2021
        'Rivian', 'Coinbase', 'Roblox', 'UiPath', 'Bumble', 'AppLovin',
        'Robinhood', 'Toast', 'Warby Parker', 'Freshworks', 'GitLab',
        'Affirm', 'SoFi', 'Marqeta', 'ZipRecruiter', 'monday.com',
        'Sprinklr', 'SentinelOne', 'Kaltura', 'PowerSchool', 'Amplitude',
        
        # 2020
        'Snowflake', 'Airbnb', 'DoorDash', 'Palantir', 'Asana',
        'Unity Software', 'JFrog', 'BigCommerce', 'Sumo Logic', 'Lemonade',
        'nCino', 'Vroom', 'Shift4 Payments', 'Jamf', 'C3.ai',
        'ZoomInfo', 'Datto', 'Corsair Gaming', 'ContextLogic (Wish)',
        
        # 2019
        'Uber', 'Lyft', 'Slack', 'Pinterest', 'Zoom', 'CrowdStrike',
        'Datadog', 'PagerDuty', 'Elastic', 'Fastly', 'Tradeweb',
        'Cloudflare', 'Dynatrace', 'Peloton', 'Chewy', 'Change Healthcare',
        'Tufin', 'Revolve', 'Jumia', 'Beyond Meat', 'Fiverr',
        
        # 2018
        'Spotify', 'Dropbox', 'DocuSign', 'SmartSheet', 'Zuora',
        'Carbon Black', 'Pivotal Software', 'Zscaler', 'iQIYI',
        'Tencent Music', 'Nio', 'Farfetch', 'MongoDB', 'SurveyMonkey'
    ],
    
    'Ticker': [
        # 2024
        'RDDT', 'ALAB', 'RBRK', 'IBTA', 'TEM',
        
        # 2023
        'ARM', 'CART', 'KVYO', 'BIRK', 'NXT',
        
        # 2022
        'TPG', 'CRDO',
        
        # 2021
        'RIVN', 'COIN', 'RBLX', 'PATH', 'BMBL', 'APP',
        'HOOD', 'TOST', 'WRBY', 'FRSH', 'GTLB',
        'AFRM', 'SOFI', 'MQ', 'ZIP', 'MNDY',
        'CXM', 'S', 'KLTR', 'PWSC', 'AMPL',
        
        # 2020
        'SNOW', 'ABNB', 'DASH', 'PLTR', 'ASAN',
        'U', 'FROG', 'BIGC', 'SUMO', 'LMND',
        'NCNO', 'VRM', 'FOUR', 'JAMF', 'AI',
        'ZI', 'DATTO', 'CRSR', 'WISH',
        
        # 2019
        'UBER', 'LYFT', 'WORK', 'PINS', 'ZM', 'CRWD',
        'DDOG', 'PD', 'ESTC', 'FSLY', 'TW',
        'NET', 'DT', 'PTON', 'CHWY', 'CHNG',
        'TUFN', 'RVLV', 'JMIA', 'BYND', 'FVRR',
        
        # 2018
        'SPOT', 'DBX', 'DOCU', 'SMAR', 'ZUO',
        'CBLK', 'PVTL', 'ZS', 'IQ',
        'TME', 'NIO', 'FTCH', 'MDB', 'SVMK'
    ],
    
    'IPO_Date': [
        # 2024
        '2024-03-21', '2024-03-20', '2024-04-27', '2024-04-18', '2024-06-14',
        
        # 2023
        '2023-09-14', '2023-09-19', '2023-09-20', '2023-10-11', '2023-02-09',
        
        # 2022
        '2022-01-13', '2022-02-03',
        
        # 2021
        '2021-11-10', '2021-04-14', '2021-03-10', '2021-04-21', '2021-02-11', '2021-04-12',
        '2021-07-29', '2021-09-22', '2021-09-29', '2021-09-22', '2021-10-14',
        '2021-01-13', '2021-01-07', '2021-06-09', '2021-05-26', '2021-06-10',
        '2021-06-23', '2021-06-30', '2021-07-21', '2021-08-11', '2021-09-28',
        
        # 2020
        '2020-09-16', '2020-12-10', '2020-12-09', '2020-09-30', '2020-09-30',
        '2020-09-18', '2020-09-16', '2020-08-05', '2020-09-17', '2020-07-02',
        '2020-07-14', '2020-06-09', '2020-06-09', '2020-07-22', '2020-12-09',
        '2020-06-04', '2020-10-21', '2020-09-23', '2020-12-16',
        
        # 2019
        '2019-05-10', '2019-03-29', '2019-06-20', '2019-04-18', '2019-04-18', '2019-06-12',
        '2019-09-19', '2019-04-11', '2019-10-03', '2019-05-17', '2019-04-04',
        '2019-09-13', '2019-08-01', '2019-09-26', '2019-06-14', '2019-06-27',
        '2019-04-11', '2019-06-07', '2019-04-12', '2019-05-02', '2019-06-13',
        
        # 2018
        '2018-04-03', '2018-03-23', '2018-04-27', '2018-04-27', '2018-04-12',
        '2018-05-03', '2018-04-20', '2018-03-16', '2018-03-29',
        '2018-12-12', '2018-09-12', '2018-09-21', '2018-10-11', '2018-09-26'
    ],
    
    'Sector': [
        # 2024
        'Social Media', 'Semiconductors', 'Cloud Security', 'E-commerce', 'Healthcare Tech',
        
        # 2023
        'Semiconductors', 'E-commerce', 'Marketing Tech', 'Consumer', 'Clean Energy',
        
        # 2022
        'Financial Services', 'Semiconductors',
        
        # 2021
        'Automotive', 'Fintech', 'Gaming', 'Enterprise Software', 'Social', 'Mobile Gaming',
        'Fintech', 'Payments', 'E-commerce', 'Enterprise Software', 'DevOps',
        'Fintech', 'Fintech', 'Payments', 'HR Tech', 'Productivity',
        'Marketing', 'Cybersecurity', 'Video', 'Education Tech', 'Analytics',
        
        # 2020
        'Cloud Data', 'Travel', 'Delivery', 'Big Data', 'Productivity',
        'Gaming', 'DevOps', 'E-commerce', 'Cloud', 'Insurtech',
        'Fintech', 'E-commerce', 'Payments', 'Device Management', 'AI',
        'Sales Tech', 'IT Management', 'Gaming', 'E-commerce',
        
        # 2019
        'Rideshare', 'Rideshare', 'Enterprise Collaboration', 'Social Media', 'Video Conferencing', 'Cybersecurity',
        'Cloud Monitoring', 'DevOps', 'Search/Analytics', 'Edge Computing', 'Financial Tech',
        'Edge Computing', 'Application Performance', 'Fitness', 'E-commerce', 'Healthcare IT',
        'Cybersecurity', 'Fashion E-commerce', 'E-commerce', 'Food Tech', 'Freelance Platform',
        
        # 2018
        'Music Streaming', 'Cloud Storage', 'E-signature', 'Collaboration', 'Subscription Billing',
        'Cybersecurity', 'Cloud Platform', 'Cloud Security', 'Video Streaming',
        'Music Streaming', 'Electric Vehicles', 'Luxury E-commerce', 'Database', 'Survey Software'
    ]
}

# Create DataFrame
df_ipos = pd.DataFrame(tech_ipos_data)

# Convert date
df_ipos['IPO_Date'] = pd.to_datetime(df_ipos['IPO_Date'])

# Calculate lockup expiration (180 days standard)
df_ipos['Lockup_Expiration'] = df_ipos['IPO_Date'] + pd.Timedelta(days=180)

# Add year for easier filtering
df_ipos['IPO_Year'] = df_ipos['IPO_Date'].dt.year

print("="*80)
print("CURATED TECH IPO DATASET")
print("="*80)

print(f"\n📊 Dataset Summary:")
print(f"   Total IPOs: {len(df_ipos)}")
print(f"   Date range: {df_ipos['IPO_Date'].min().date()} to {df_ipos['IPO_Date'].max().date()}")
print(f"   Sectors: {df_ipos['Sector'].nunique()}")

print(f"\n📈 IPOs by Year:")
print(df_ipos['IPO_Year'].value_counts().sort_index())

print(f"\n🏢 Sample IPOs:")
print(df_ipos[['Company', 'Ticker', 'IPO_Date', 'Lockup_Expiration', 'Sector']].head(15))

# Save
output_path = f'{output_dir}/tech_ipos_curated.csv'
df_ipos.to_csv(output_path, index=False)
print(f"\n💾 Saved to: {output_path}")

print(f"\n✅ Ready for stock price download!")

CURATED TECH IPO DATASET

📊 Dataset Summary:
   Total IPOs: 87
   Date range: 2018-03-16 to 2024-06-14
   Sectors: 58

📈 IPOs by Year:
IPO_Year
2018    14
2019    21
2020    19
2021    21
2022     2
2023     5
2024     5
Name: count, dtype: int64

🏢 Sample IPOs:
             Company Ticker   IPO_Date Lockup_Expiration              Sector
0             Reddit   RDDT 2024-03-21        2024-09-17        Social Media
1        Astera Labs   ALAB 2024-03-20        2024-09-16      Semiconductors
2             Rubrik   RBRK 2024-04-27        2024-10-24      Cloud Security
3             Ibotta   IBTA 2024-04-18        2024-10-15          E-commerce
4          Tempus AI    TEM 2024-06-14        2024-12-11     Healthcare Tech
5       Arm Holdings    ARM 2023-09-14        2024-03-12      Semiconductors
6          Instacart   CART 2023-09-19        2024-03-17          E-commerce
7            Klaviyo   KVYO 2023-09-20        2024-03-18      Marketing Tech
8        Birkenstock   BIRK 2023-10-11      

# Stock price download

In [3]:
# Load IPO list
df_ipos = pd.read_csv('../data/processed/tech_ipos_curated.csv', parse_dates=['IPO_Date', 'Lockup_Expiration'])

print(f"📥 Downloading stock prices for {len(df_ipos)} IPOs...")

# Store each company's data separately, then concat
stock_data_list = []

for idx, row in df_ipos.iterrows():
    ticker = row['Ticker']
    ipo_date = row['IPO_Date']
    company = row['Company']
    
    # Download from IPO date to +365 days
    start_date = ipo_date
    end_date = ipo_date + timedelta(days=365)
    
    try:
        print(f"   {ticker:6s} ({company:30s})... ", end='')
        
        # Download for THIS TICKER ONLY
        stock = yf.download(
            ticker,
            start=start_date,
            end=end_date,
            progress=False,
            auto_adjust=False  # Keep raw OHLCV
        )
        
        if len(stock) > 0:
            # Reset index to make Date a column
            stock = stock.reset_index()
            
            # CRITICAL: Flatten multi-index columns if present
            if isinstance(stock.columns, pd.MultiIndex):
                stock.columns = stock.columns.get_level_values(0)
            
            # Add metadata columns
            stock['Ticker'] = ticker
            stock['Company'] = company
            stock['IPO_Date'] = ipo_date
            stock['Days_Since_IPO'] = (stock['Date'] - ipo_date).dt.days
            
            # Reorder columns
            cols = ['Date', 'Ticker', 'Company', 'IPO_Date', 'Days_Since_IPO', 
                    'Open', 'High', 'Low', 'Close', 'Volume']
            stock = stock[cols]
            
            stock_data_list.append(stock)
            print(f"✅ {len(stock)} days")
        else:
            print(f"❌ No data")
            
    except Exception as e:
        print(f"❌ Error: {str(e)[:50]}")
    
    # Rate limiting
    if (idx + 1) % 10 == 0:
        print(f"\n   Processed {idx + 1}/{len(df_ipos)}...\n")
        time.sleep(2)

# Combine all into single DataFrame (LONG FORMAT)
if len(stock_data_list) > 0:
    df_stock_prices = pd.concat(stock_data_list, ignore_index=True)
    
    print(f"\n✅ Stock prices downloaded:")
    print(f"   Companies: {df_stock_prices['Ticker'].nunique()}")
    print(f"   Total observations: {len(df_stock_prices):,}")
    print(f"   Columns: {df_stock_prices.columns.tolist()}")
    
    # Verify data types
    print(f"\n📊 Data types:")
    print(df_stock_prices.dtypes)
    
    # Sample check
    print(f"\n🔍 Sample (first 5 rows):")
    print(df_stock_prices.head())
    
    # Save
    output_path = '../data/processed/stock_prices_ipo.csv'
    df_stock_prices.to_csv(output_path, index=False)
    print(f"\n💾 Saved to: {output_path}")
else:
    print(f"\n❌ No stock data collected")

📥 Downloading stock prices for 87 IPOs...
   RDDT   (Reddit                        )... ✅ 250 days
   ALAB   (Astera Labs                   )... ✅ 250 days
   RBRK   (Rubrik                        )... ✅ 249 days
   IBTA   (Ibotta                        )... ✅ 250 days
   TEM    (Tempus AI                     )... ✅ 250 days
   ARM    (Arm Holdings                  )... ✅ 251 days
   CART   (Instacart                     )... ✅ 251 days
   KVYO   (Klaviyo                       )... ✅ 251 days
   BIRK   (Birkenstock                   )... ✅ 251 days
   NXT    (Nextracker                    )... ✅ 251 days

   Processed 10/87...

   TPG    (TPG                           )... ✅ 251 days
   CRDO   (Credo Technology              )... ✅ 251 days
   RIVN   (Rivian                        )... ✅ 252 days
   COIN   (Coinbase                      )... ✅ 254 days
   RBLX   (Roblox                        )... ✅ 253 days
   PATH   (UiPath                        )... ✅ 253 days
   BMBL   (Bumble     


1 Failed download:
['PWSC']: YFTzMissingError('possibly delisted; no timezone found')


❌ No data
   AMPL   (Amplitude                     )... ✅ 252 days
   SNOW   (Snowflake                     )... ✅ 252 days
   ABNB   (Airbnb                        )... ✅ 252 days
   DASH   (DoorDash                      )... ✅ 252 days
   PLTR   (Palantir                      )... ✅ 252 days
   ASAN   (Asana                         )... ✅ 252 days
   U      (Unity Software                )... ✅ 252 days
   FROG   (JFrog                         )... ✅ 252 days

   Processed 40/87...

   BIGC   (BigCommerce                   )... 

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: BIGC"}}}

1 Failed download:
['BIGC']: YFTzMissingError('possibly delisted; no timezone found')


❌ No data
   SUMO   (Sumo Logic                    )... 

...

❌ No data

   Processed 80/87...

   ZS     (Zscaler                       )... ✅ 251 days
   IQ     (iQIYI                         )... ✅ 251 days
   TME    (Tencent Music                 )... ✅ 252 days
   NIO    (Nio                           )... ✅ 251 days
   FTCH   (Farfetch                      )... 


1 Failed download:
['FTCH']: YFTzMissingError('possibly delisted; no timezone found')


❌ No data
   MDB    (MongoDB                       )... ✅ 251 days
   SVMK   (SurveyMonkey                  )... 


1 Failed download:
['SVMK']: YFTzMissingError('possibly delisted; no timezone found')


❌ No data

✅ Stock prices downloaded:
   Companies: 71
   Total observations: 17,874
   Columns: ['Date', 'Ticker', 'Company', 'IPO_Date', 'Days_Since_IPO', 'Open', 'High', 'Low', 'Close', 'Volume']

📊 Data types:
Price
Date              datetime64[ns]
Ticker                    object
Company                   object
IPO_Date          datetime64[ns]
Days_Since_IPO             int64
Open                     float64
High                     float64
Low                      float64
Close                    float64
Volume                     int64
dtype: object

🔍 Sample (first 5 rows):
Price       Date Ticker Company   IPO_Date  Days_Since_IPO       Open  \
0     2024-03-21   RDDT  Reddit 2024-03-21               0  47.000000   
1     2024-03-22   RDDT  Reddit 2024-03-21               1  48.880001   
2     2024-03-25   RDDT  Reddit 2024-03-21               4  47.090000   
3     2024-03-26   RDDT  Reddit 2024-03-21               5  67.709999   
4     2024-03-27   RDDT  Reddit 2024-03-21   

# MARKET ADJUSTMENT
Calculate Abnormal Returns<br>
Remove COVID/macro effects by adjusting for S&P 500


In [ ]:
print("="*80)
print("MARKET ADJUSTMENT")
print("="*80)

# Load IPO stock data
df_stock = pd.read_csv('../data/processed/stock_prices_ipo.csv', parse_dates=['Date', 'IPO_Date'])

print(f"\n📊 Stock data loaded:")
print(f"   Companies: {df_stock['Ticker'].nunique()}")
print(f"   Observations: {len(df_stock):,}")

# ============================================================================
# FIX: Convert numeric columns from string to float
# ============================================================================

print(f"\n🔧 Fixing data types...")

# Check current dtypes
print(f"   Current 'Close' dtype: {df_stock['Close'].dtype}")

# Numeric columns that should be float
numeric_cols = ['Close', 'Open', 'High', 'Low', 'Volume']

for col in numeric_cols:
    if col in df_stock.columns:
        # Convert to numeric, coercing errors to NaN
        df_stock[col] = pd.to_numeric(df_stock[col], errors='coerce')
        print(f"   ✅ Converted '{col}' to numeric")

# Verify
print(f"   New 'Close' dtype: {df_stock['Close'].dtype}")

# Check for any conversion issues
null_close = df_stock['Close'].isna().sum()
if null_close > 0:
    print(f"   ⚠️  Warning: {null_close} null values in Close after conversion")

# ============================================================================
# Continue with market adjustment
# ============================================================================

# Download S&P 500 benchmark
print(f"\n📥 Downloading S&P 500 benchmark...")

spy = yf.download(
    'SPY',
    start=df_stock['Date'].min(),
    end=df_stock['Date'].max(),
    progress=False
)

print(f"   Raw SPY data shape: {spy.shape}")

# FIX: Flatten multi-index columns if present
if isinstance(spy.columns, pd.MultiIndex):
    spy.columns = spy.columns.get_level_values(0)
    print(f"   ✅ Flattened multi-index columns")

# Calculate daily returns
spy['SPY_Return'] = spy['Close'].pct_change() * 100

# Reset index
spy_returns = spy[['SPY_Return']].reset_index()

# Ensure Date column name
if 'index' in spy_returns.columns:
    spy_returns.rename(columns={'index': 'Date'}, inplace=True)

print(f"   ✅ S&P 500 data: {len(spy_returns)} trading days")

# Merge with IPO data
print(f"\n🔗 Merging with stock data...")

df_stock = df_stock.merge(spy_returns, on='Date', how='left')

print(f"   ✅ Merged successfully")

# Calculate daily stock returns
print(f"\n📈 Calculating stock returns...")
df_stock = df_stock.sort_values(['Ticker', 'Date'])

# NOW this will work because Close is numeric
df_stock['Stock_Return'] = df_stock.groupby('Ticker')['Close'].pct_change() * 100

# Calculate ABNORMAL return (stock - market)
df_stock['Abnormal_Return'] = df_stock['Stock_Return'] - df_stock['SPY_Return']

print(f"   ✅ Abnormal returns calculated")

# Calculate cumulative abnormal return from IPO
print(f"\n📊 Calculating cumulative abnormal returns...")

def calculate_cumulative(group):
    """Cumulative abnormal return from IPO date"""
    group = group.sort_values('Days_Since_IPO')
    group['Cum_Abnormal_Return'] = group['Abnormal_Return'].fillna(0).cumsum()
    return group

df_stock = df_stock.groupby('Ticker').apply(calculate_cumulative).reset_index(drop=True)

print(f"   ✅ Cumulative abnormal returns calculated")

# Summary statistics
print(f"\n📊 Summary Statistics:")
print(f"   Total observations: {len(df_stock):,}")
print(f"   Non-null abnormal returns: {df_stock['Abnormal_Return'].notna().sum():,}")
print(f"   Mean daily abnormal return: {df_stock['Abnormal_Return'].mean():.4f}%")
print(f"   Std daily abnormal return: {df_stock['Abnormal_Return'].std():.4f}%")

# Check Day 180 specifically
day_180_data = df_stock[df_stock['Days_Since_IPO'] == 180]
if len(day_180_data) > 0:
    median_180 = day_180_data['Cum_Abnormal_Return'].median()
    mean_180 = day_180_data['Cum_Abnormal_Return'].mean()
    print(f"   Median cum. abnormal return at Day 180: {median_180:.2f}%")
    print(f"   Mean cum. abnormal return at Day 180: {mean_180:.2f}%")

# Sample data check
print(f"\n🔍 Sample data (first company, first 5 days):")
first_ticker = df_stock['Ticker'].iloc[0]
sample = df_stock[df_stock['Ticker'] == first_ticker].head(5)
print(sample[['Ticker', 'Date', 'Days_Since_IPO', 'Close', 'Stock_Return', 'SPY_Return', 'Abnormal_Return']].to_string(index=False))

# Check for missing values
missing_abn = df_stock['Abnormal_Return'].isna().sum()
pct_missing = (missing_abn / len(df_stock)) * 100
print(f"\n📊 Data quality:")
print(f"   Missing abnormal returns: {missing_abn} ({pct_missing:.2f}%)")
print(f"   (First day of each IPO expected to be null)")

# Save market-adjusted data
output_path = '../data/processed/stock_prices_ipo_adjusted.csv'
df_stock.to_csv(output_path, index=False)

print(f"\n💾 Saved: {output_path}")

/var/folders/s9/3vxnll4s24b0zx1_rpkbk7r40000gp/T/ipykernel_15167/3506242703.py:45: FutureWarning: YF.download() has changed argument auto_adjust default to True
  spy = yf.download(


MARKET ADJUSTMENT

📊 Stock data loaded:
   Companies: 71
   Observations: 17,874

🔧 Fixing data types...
   Current 'Close' dtype: float64
   ✅ Converted 'Close' to numeric
   ✅ Converted 'Open' to numeric
   ✅ Converted 'High' to numeric
   ✅ Converted 'Low' to numeric
   ✅ Converted 'Volume' to numeric
   New 'Close' dtype: float64

📥 Downloading S&P 500 benchmark...
   Raw SPY data shape: (1821, 5)
   ✅ Flattened multi-index columns
   ✅ S&P 500 data: 1821 trading days

🔗 Merging with stock data...
   ✅ Merged successfully

📈 Calculating stock returns...
   ✅ Abnormal returns calculated

📊 Calculating cumulative abnormal returns...
   ✅ Cumulative abnormal returns calculated

📊 Summary Statistics:
   Total observations: 17,874
   Non-null abnormal returns: 17,802
   Mean daily abnormal return: 0.0254%
   Std daily abnormal return: 4.4802%
   Median cum. abnormal return at Day 180: -8.03%
   Mean cum. abnormal return at Day 180: -5.07%

🔍 Sample data (first company, first 5 days):
Ti

/var/folders/s9/3vxnll4s24b0zx1_rpkbk7r40000gp/T/ipykernel_15167/3506242703.py:99: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_stock = df_stock.groupby('Ticker').apply(calculate_cumulative).reset_index(drop=True)
